In [1]:
import secrets
import hashlib

In [2]:
def random_binary_generator(n):
    # Gets integer n,
    # Generates n random bits and returns it 
    # (Without the 0b)
    random_bits = secrets.randbits(n)
    binary_string = bin(random_bits)[2:]
    binary_string = binary_string.zfill(n)

    return binary_string

In [3]:
def count_one_elements(binary_number):
    # Gets a binary number,
    # Counts '1' elements in binary number and returns it
    count = 0
    for digit in binary_number:
        if digit == '1':
            count += 1
    return count

def rotate_left(binary_number, shift_amount):
    # Gets a binary number and shitf amount,
    # Returns rotated number in binary
    rotated_binary = binary_number[shift_amount:] + binary_number[:shift_amount]
    return rotated_binary

def rotate_right(binary_number, shift_amount):
    # Gets a binary number and shitf amount,
    # Returns rotated number in binary
    rotated_binary = binary_number[-shift_amount:] + binary_number[:-shift_amount]
    return rotated_binary

In [4]:
def binary_xor(*args):
    # Gets anu amount of binary numbers, 
    # (converts them to same lenth if they are not)
    # Calculates XOR of them and then returns it

    max_length = max(len(binary_string) for binary_string in args)
    padded_args = [binary_string.zfill(max_length) for binary_string in args]
    xor_binary_string = ''.join(str(sum(int(bit) for bit in bits) % 2) for bits in zip(*padded_args))

    return xor_binary_string

In [5]:
def hash_binary(binary_string):
    # Gets a binary number,
    # returns hash of it in hex
    decimal_number = int(binary_string, 2)
    byte_data = decimal_number.to_bytes((decimal_number.bit_length() + 7) // 8, byteorder='big')
    sha1_hash = hashlib.sha1(byte_data).hexdigest()

    sha1_hash = '0x' + sha1_hash
    return sha1_hash

## IDs of nodes

In [6]:
IDNt_bin = random_binary_generator(160)
IDNr_bin = random_binary_generator(160)
IDNs_bin = random_binary_generator(160)
BalN_bin = random_binary_generator(160)
BS_bin = random_binary_generator(160)

print(f"IDNt_bin = '{IDNt_bin}'")
print(f"IDNr_bin = '{IDNr_bin}'")
print(f"IDNs_bin = '{IDNs_bin}'")
print(f"BalN_bin = '{BalN_bin}'")
print(f"BS_bin = '{BS_bin}'")

IDNt_bin = '0010101111000011001000000010100100110011000011001010000101010011001010100101110001011000101100111110110010001111100011000111101111111111101100101011110001110010'
IDNr_bin = '0010110111001000001111000010111111100010010100100111101001101110100101011000100001010100000110000001011010010101001010110110101010110000011011110000111001110000'
IDNs_bin = '1010001111000111111001111100111111001000111110101010001011000001110111000101011110101110100001011000111001000000001101101101111011110001010001010101000111000001'
BalN_bin = '1000110111111000011000100100101111111011001101100010000110111101101011110000110001001001001011011101100000010000001001101111111000010110111100000001101010011011'
BS_bin = '1001101011100011111010110101101111000000010110111111101011111100110100001001100101010111010101010010110001010010010111100110101101100000011000011001111110000000'


## Msg 1

In [7]:
R0_bin = random_binary_generator(160)
TIR_bin = random_binary_generator(32)

In [25]:
R0_bin

'1110101000001110110000101000100111001011111011000110010101110101010010111101001011111111010011001110001000001110101110101100101000100100101111001010101111001111'

In [8]:
# PR
# R0_bin ^ IDNt_bin ^ TIR_bin
PR_1_bin = binary_xor(R0_bin, IDNt_bin, TIR_bin)

# TIR_bin ^ IDNt_bin
PR_2_bin = binary_xor(TIR_bin, IDNt_bin)

PR_bin = rotate_left(PR_1_bin ,count_one_elements(PR_2_bin))

In [9]:
# PR_bin ^ IDNt_bin ^ TIR_bin
pre_CHR_bin = binary_xor(PR_bin, IDNt_bin, TIR_bin)
CHR_hex = hash_binary(pre_CHR_bin)

In [10]:
msg1 = {
    "PR_bin": PR_bin,
    "CHR_hex": CHR_hex,
    "TIR_bin": TIR_bin
}

print(f"PR_bin = '{PR_bin}'")
print(f"CHR_hex = '{CHR_hex}'")
print(f"TIR_bin = '{TIR_bin}'")

PR_bin = '0110001110101001111111111100001110100000010011011010110001100011001011001010101100111011101100000111001101111000101010000011111000111000001100010000100110011000'
CHR_hex = '0x34873dbe9387e9d312bc5e6940bad2dafd175b5c'
TIR_bin = '01010111101111001011101101010011'


## Msg 3

In [18]:
# Should be sent from Tag => Reader
# Msg 2
CHT_hex = '0x62a263ee93426984701a50427ad9f99b615b7eff'
AthR_hex = '0x20bc909d7021476867db51928b99fa4b81301574'
PT_bin = '0010101010001110010010110110001001110100011000001010010111110111011111101000010000101010010011100100100100101010001100000001100010110110001111011010010010111100'
TIT_bin = '01101011000101100010101010001011'

In [19]:
TIR_prime_bin = random_binary_generator(32)
RC_bin = random_binary_generator(160)
Rd_bin = random_binary_generator(160)

In [20]:
# PP
# TIR_prime_bin ^ IDNs_bin ^ Rd_bin
PP_bin = binary_xor(TIR_prime_bin, IDNs_bin, Rd_bin)

In [21]:
# YRS
# IDNs_bin || BS_bin || IDNr_bin
pre_YRS = IDNs_bin + BS_bin + IDNr_bin
YRS_hex = hash_binary(pre_YRS)

In [22]:
# PQ
PQ_bin = binary_xor(bin(int(YRS_hex, 16))[2:], Rd_bin)

In [23]:
conct_for_reader_ckeck = Rd_bin + TIR_prime_bin
pre_Reader_check = binary_xor(RC_bin, IDNs_bin, BalN_bin, conct_for_reader_ckeck)
Reader_check_hex = hash_binary(pre_Reader_check)

In [24]:
msg3 = {
    "PP_bin" : PP_bin,
    "PQ_bin" : PQ_bin,
    "Reader_check_hex" : Reader_check_hex,
    "TIR_prime_bin" : TIR_prime_bin
}

print(f"PP_bin = '{PP_bin}'")
print(f"PQ_bin = '{PQ_bin}'")
print(f"Reader_check_hex = '{Reader_check_hex}'")
print(f"TIR_prime_bin = '{TIR_prime_bin}'")

PP_bin = '0101001111010001001010111111011101000110110100000011000100001110010111000001100110101110100000101010011101101010101000100101110111010101110001000110011101001101'
PQ_bin = '0111101000101100110101010010110011111110100010001011101000101001001010101100011110100100110011100010011001101010000110101101001010111011101011110111110111111100'
Reader_check_hex = '0x5df5d19d9d365ce4788b762b95c789034b3cab5e'
TIR_prime_bin = '10101110111011001011000001101000'
